# First Load File

In [45]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [46]:
file_path = "TSX Data  - Sheet1.csv"
loader = CSVLoader(file_path=file_path)
data = loader.load()

# Create a Text Splitter / Chunker

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [48]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# Apply text_spliiter to your doc

In [49]:
split = text_splitter.split_documents(data)

# DB and Embeddign Model

In [51]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [22]:
embedder = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

# Store vector into the db

In [23]:
store = Chroma.from_documents(
    documents=split,
    embedding=embedder,
    collection_name="TSX",
    persist_directory="TSX",
)

# Creating one retriever from the db

In [52]:
retriver = store.as_retriever(
    search_type = "similarity",
    search_kwargs={
        "k": 5,
    }
)

# Creating a rag chain 

In [32]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import ChatGoogleGenerativeAI


# Chat Model

In [53]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.5, max_tokens=500)

In [54]:
system_prompt = (
    "You are a helpful assistant that answers questions about the TSX data Of CSV file. {context}"
)

# Chat Model Template

In [55]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]

)

# Creating a chain

In [56]:
question_answer_chain = create_stuff_documents_chain(llm, prompt) # connect llm and prompt

In [58]:
retrieve_chain = create_retrieval_chain(  # connect retriever/db and chain
    retriver,
    question_answer_chain,
)

In [62]:
query = "Who has a guptin of 1743?"

In [63]:
response = retrieve_chain.invoke({"input": query})
print(response['answer'])

David Garcia has a guptin of 1743.
